SPDX-License-Identifier: GPL-2.0-or-later OR AGPL-3.0-or-later OR CERN-OHL-S-2.0+

# Example notebook with a FreePDK45 inverter

This noebook show the design, layout and verification of a minimum area balances inverter.

It's an example of the different aspects of the PDKMaster based design flow the is envisioned for the Chips4Makers. Amongst them:

* Design based on single file with technology description in python, e.g. `c4m.pdk.freepdk45.pdkmaster`.   
  This module contains all information for the (non-manufacturable) FreePDK45 technology needed to be able to make design in that node using the PDKMaster framework.
* Integrated interactive circuit design, simulation and layout. This is in contract with other EDA flows where schematic capture and simulation are separated from circuit layout.
* Design rule driven parameterized layout generation with easy portability between technologies.
* Parameterized circuit definition and simulation.
* Compatible with numpy data analysis and matplotlib plotting.

That said this is currently based on the PDKMaster with a highly unstable API with at the moment no backwards whatsoever. Big if not fully rework of the API is still planned. This includes:

* use klayout for layout engine and not shapely
* better integration of circuit simulation with more unified API
* introduction of signal types like analog signals, digital signals or DC lines etc.

# Setup

This notebook needs a kernel with the C4M FreePDK45 PDK installed and also pyspice for simulation.

In [ ]:
from matplotlib import pyplot as plt, ticker
import numpy as np

from pdkmaster.design import layout as lay, circuit as ckt, library as lbry
from c4m.pdk.freepdk45 import tech, layoutfab, cktfab, pyspicefab, plotter
from c4m.PySpice.Unit import *

# Parametric inverter code

First is that we can define a cell to represent the inverter. The parameters are the width of the nmos and the pmos. Minimum l will be used for both transistors and a miminal layout will be generated.

## InverterLibrary

Currently a Cell needs to be always be created in a library. Therefor we provide a dummy library. This requirement may be revised in the future.

In [ ]:
class InverterLibrary(lbry.Library):
    def __init__(self):
        super().__init__("InverterLibrary",
            tech=tech, cktfab=cktfab, layoutfab=layoutfab
        )
invlib = InverterLibrary()

## Inverter

In [ ]:
class Inverter(lbry._Cell):
    def __init__(self, *, nmos, w_n, pmos, w_p):
        super().__init__(invlib, "Inverter")

        l_n = nmos.computed.min_l
        l_p = pmos.computed.min_l

        #
        # Create the circuit
        #
        inv = self.new_circuit()
        n1 = inv.new_instance("n1", nmos, l=l_n, w=w_n)
        p1 = inv.new_instance("p1", pmos, l=l_p, w=w_p)
        # n1 = inv.new_instance("n1", nmos, l=l_n, w=w_n, activeimplant_enclosure=0.020)
        # p1 = inv.new_instance("p1", pmos, l=l_p, w=w_p, activeimplant_enclosure=0.020)
        in_ = inv.new_net("IN", external=True)
        out = inv.new_net("OUT", external=True)
        vdd = inv.new_net("VDD", external=True)
        gnd = inv.new_net("GND", external=True)
        in_.childports += (n1.ports.gate, p1.ports.gate)
        gnd.childports += (n1.ports.sourcedrain1, n1.ports.bulk)
        vdd.childports += (p1.ports.sourcedrain1, p1.ports.bulk)
        out.childports += (n1.ports.sourcedrain2, p1.ports.sourcedrain2)

        #
        # Create layout
        #
        contact = tech.primitives.contact
        active = tech.primitives.active
        poly = tech.primitives.poly
        nimplant = tech.primitives.nimplant
        pimplant = tech.primitives.pimplant
        nwell = tech.primitives.nwell
        pwell = tech.primitives.pwell
        metal1 = tech.primitives.metal1

        # TODO: Use packer to derive minimal placement of p transistor
        impl_n_enc = nmos.min_gateimplant_enclosure[0] # This is not generic yet
        impl_p_enc = pmos.min_gateimplant_enclosure[0] # This is not generic yet
        y_p = 0.5*w_n + impl_n_enc.spec + impl_p_enc.spec + 0.5*w_p

        x_v = max(
            0.5*l_n + nmos.computed.min_contactgate_space,
            0.5*l_p + pmos.computed.min_contactgate_space,
        ) + 0.5*contact.width

        layouter = self.new_circuitlayouter()

        # gnd tap to pwell
        layouter.add_wire(
            net=gnd, wire=contact, x=0, y=-0.2,
            bottom=active, bottom_implant=pimplant, bottom_well=pwell,
            top_width=0.4,
        )
        # vdd tap to nwell
        layouter.add_wire(
            net=vdd, wire=contact, x=0, y=0.5,
            bottom=active, bottom_implant=nimplant, bottom_well=nwell,
            top_width=0.4,
        )

        # gnd contact to source of nmos
        layouter.add_wire(
            net=gnd, well_net=gnd, wire=contact, x=-x_v, y=0.0,
            bottom=active, bottom_implant=nimplant, bottom_well=pwell,
            bottom_height=w_n,
        )
        # vdd contact to source of pmos
        layouter.add_wire(
            net=vdd, well_net=vdd, wire=contact, x=-x_v, y=y_p,
            bottom=active, bottom_implant=pimplant, bottom_well=nwell,
            bottom_height=w_p,
        )

        # The two transistors
        layouter.place(n1, x=0.0, y=0.0)
        layouter.place(p1, x=0.0, y=y_p)

        # out contact to drain of nmos
        layouter.add_wire(
            net=out, well_net=gnd, wire=contact, x=x_v, y=0.0,
            bottom=active, bottom_implant=nimplant, bottom_well=pwell,
            bottom_height=w_n,
        )
        # out contact to drain of pmos
        layouter.add_wire(
            net=out, well_net=vdd, wire=contact, x=x_v, y=y_p,
            bottom=active, bottom_implant=pimplant, bottom_well=nwell,
            bottom_height=w_p,
        )

        layouter.connect(masks=(
            nwell.mask, pwell.mask, poly.mask, metal1.mask,
        ))

# Design balanced inverter

We use the general threshold transistors @ 1.0V vdd

In [ ]:
nmos = tech.primitives.nmos_vtg
pmos = tech.primitives.pmos_vtg

vdd = u_V(1.0)

Support functions for simulating Ion

In [ ]:
def sim_ion_nmos(*, w_n):
    dut = cktfab.new_circuit("nmos")
    ntrans = dut.new_instance("n", nmos, l=nmos.computed.min_l, w=w_n)
    g = dut.new_net("g", external=True, childports=ntrans.ports.gate)
    s = dut.new_net("s", external=True, childports=ntrans.ports.sourcedrain1)
    d = dut.new_net("d", external=True, childports=ntrans.ports.sourcedrain2)
    b = dut.new_net("b", external=True, childports=ntrans.ports.bulk)

    tb = pyspicefab.new_pyspicecircuit(corner="NOM", top=dut, title="nmos Ion")
    tb.V("gate", "g", tb.gnd, vdd)
    tb.V("drain", "d", tb.gnd, vdd)
    tb.V("source", "s", tb.gnd, u_V(0))
    tb.V("bulk", "b", tb.gnd, u_V(0))

    sim = tb.simulator(temperature=u_Degree(25))
    op = sim.operating_point()
    return -op.vdrain[0].value

def sim_ion_pmos(*, w_p):
    dut = cktfab.new_circuit("pmos")
    ntrans = dut.new_instance("p", pmos, l=pmos.computed.min_l, w=w_p)
    g = dut.new_net("g", external=True, childports=ntrans.ports.gate)
    s = dut.new_net("s", external=True, childports=ntrans.ports.sourcedrain1)
    d = dut.new_net("d", external=True, childports=ntrans.ports.sourcedrain2)
    b = dut.new_net("b", external=True, childports=ntrans.ports.bulk)

    tb = pyspicefab.new_pyspicecircuit(corner="NOM", top=dut, title="pmos Ion")
    tb.V("gate", "g", tb.gnd, u_V(0))
    tb.V("drain", "d", tb.gnd, vdd)
    tb.V("source", "s", tb.gnd, u_V(0))
    tb.V("bulk", "b", tb.gnd, vdd)

    sim = tb.simulator(temperature=u_Degree(25))
    op = sim.operating_point()
    return -op.vdrain[0].value


We use minimal w for the nmos and determine the w for pmos so the drive strength is the same. This is done by simulation.

In [ ]:
w_n = nmos.computed.min_w
ion_nmos = 1e6*sim_ion_nmos(w_n=w_n)

w_ps = pmos.computed.min_w*np.arange(1.0, 3.01, 0.25)
ions_pmos = tuple(1e6*sim_ion_pmos(w_p=w_p) for w_p in w_ps)
w_p_int = np.interp(ion_nmos, ions_pmos, w_ps)
w_p = tech.on_grid(w_p_int, mult=2) # On even mutiple of grid
ion_pmos = 1e6*sim_ion_pmos(w_p=w_p)

plt.figure(figsize=(6, 6))
plt.plot(w_ps, ions_pmos, label="pmos")
plt.plot(w_n, ion_nmos, '*', label="$w_n$=0.09µm")
plt.plot(w_p, ion_pmos, '*', label=f"$w_p$={w_p:.2f}µm")
plt.axis([0, 0.3, 0, 180])
plt.legend()
plt.xlabel("$w$ [µm]")
plt.ylabel("$I_{on}$ [µA]")
plt.grid("on")
plt.show()

Now create the inverter and show it's layout

In [ ]:
inv = Inverter(nmos=nmos, w_n=w_n, pmos=pmos, w_p=w_p)

plt.figure(figsize=(4.0, 7.0))

plotter.plot(inv.layout)

ax = plt.gca()
ax.set_title("inverter")
ax.xaxis.set_major_locator(ticker.MultipleLocator(0.1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(0.1))
ax.axis("equal")
ax.axis([-0.25, 0.25, -0.3, 0.6])
ax.grid("on")

plt.show()

# Verify functionality

In [ ]:
def _block():
    tb = pyspicefab.new_pyspicecircuit(corner="NOM", top=inv.circuit, gnd="GND", title="Inverter verification")
    tb.V("supply", "VDD", tb.gnd, vdd)
    tb.V("in", "IN", tb.gnd, u_V(0))
    tb.C("load", "OUT", tb.gnd, u_nF(1e-3))
    
    sim = tb.simulator(temperature=u_Degree(25))
    dc = sim.dc(vin=slice(0, vdd, vdd/200))

    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    plt.plot(dc.sweep, dc.OUT)
    plt.plot(dc.sweep, dc.IN, '--')
    plt.axis("square")
    plt.xlabel("$V_{IN}$ [V]")
    plt.ylabel("$V_{OUT}$ [V]")
    plt.grid("on")

    plt.subplot(1, 2, 2)
    plt.plot(dc.sweep, -1e6*np.array(dc.vsupply))
    plt.xlabel("$V_{IN}$ [V]")
    plt.ylabel("$I_{supply}$ [µA]")
    plt.grid("on")
_block()